In [1]:
from torchtext.vocab import vocab
from collections import Counter, OrderedDict

In [2]:
import torchtext

In [3]:
from torchtext.datasets import IMDB ##IMDB 데이터셋

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from torchtext.data.utils import get_tokenizer ##tokenizer 라이브러리 
tokenizer = get_tokenizer('basic_english') ##basic_englihs 토크나이저

In [6]:
train_iter, test_iter = IMDB(root = 'torchtext.data',split = ('train', 'test')) ##훈련과 평가 데이터 분류

/opt/conda/lib/python3.7/site-packages/torch/utils/data/datapipes/utils/common.py:25: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  "Lambda function is not supported for pickle, please use "
/opt/conda/lib/python3.7/site-packages/torch/utils/data/datapipes/iter/selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "


In [7]:
def yield_tokens(iter_):
    for label, line in iter_:
        yield tokenizer(line)

In [8]:
from_iter_vocab = torchtext.vocab.build_vocab_from_iterator(yield_tokens(train_iter), specials = ['<unk>', '<BOS>', '<EOS>','<PAD>'], min_freq = 10) #단어 사전 생성

In [9]:
from_iter_vocab.get_stoi()

{'zeitgeist': 20437,
 'yves': 20435,
 'yorkshire': 20434,
 'yam': 20428,
 'wrap-up': 20423,
 'wouldnt': 20422,
 'workaholic': 20420,
 'wordless': 20419,
 'wishful': 20418,
 'winningham': 20417,
 'winch': 20415,
 'whiff': 20414,
 'warwick': 20405,
 'warring': 20404,
 'war-torn': 20402,
 'war-time': 20401,
 'walthall': 20400,
 'walkin': 20399,
 'waldo': 20398,
 'vodka': 20395,
 'vigorous': 20391,
 'vicariously': 20390,
 'verma': 20388,
 'verify': 20386,
 'updates': 20382,
 'up-to-date': 20381,
 'up-and-coming': 20380,
 'untergang': 20377,
 'unsentimental': 20376,
 'unrestrained': 20375,
 'unparalleled': 20374,
 'unneeded': 20373,
 'universities': 20372,
 'unfeeling': 20370,
 'unending': 20369,
 'undetected': 20366,
 'underscored': 20364,
 'uncharacteristically': 20360,
 'ummm': 20359,
 'udo': 20357,
 'tucked': 20356,
 'trooper': 20355,
 'trnka': 20354,
 'trashes': 20352,
 'transporting': 20351,
 'trampled': 20348,
 'trailing': 20346,
 'traders': 20345,
 'tracing': 20344,
 'tongues': 2033

In [10]:
from_iter_vocab['otu of voavl'] #out of vocabulary 일때 오류가 뜸

RuntimeError: Token otu of voavl not found and default index is not set

In [11]:
from_iter_vocab.set_default_index(from_iter_vocab['<unk>']) ## 단어 사전의 OOV단어 값을 unk 값(= 0)으로 설정 

In [12]:
from_iter_vocab['outofvoacb'] ##out of vocabulary 값이 설정 됨

0

## 단어 사전 완성.


## 문장을 토큰화

In [13]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
text_transform = lambda x: [from_iter_vocab['<BOS>']] + [from_iter_vocab[token] for token in tokenizer(x)] + [from_iter_vocab['<EOS>']]
label_transform = lambda x: 1 if x == 'pos' else 0


In [17]:
print(f"input to the text_transform:" ,"this is hello world")
print(f"output to the text_transform:", text_transform("this is hello world"))
print(f"output to tensor = ",torch.tensor(text_transform("this is hello world")))
ex_tensor = torch.tensor(text_transform("this is hello world"))
ex_tensor1 = torch.tensor(text_transform("that is not me"))
print(f"that is not me, text_transform : {ex_tensor1}")

ex_list = [] 
ex_list.append(ex_tensor)
ex_list.append(ex_tensor1)
print(pad_sequence(ex_list, padding_value = 1))

input to the text_transform: this is hello world
output to the text_transform: [1, 16, 12, 4647, 188, 2]
output to tensor =  tensor([   1,   16,   12, 4647,  188,    2])
that is not me, text_transform : tensor([ 1, 17, 12, 31, 80,  2])
tensor([[   1,    1],
        [  16,   17],
        [  12,   12],
        [4647,   31],
        [ 188,   80],
        [   2,    2]])


In [48]:
w_list = []
a = torch.tensor([1,2,3,4,5])
print(f"print a:",a)
b= torch.tensor([9,8,7,6,5])
print(f"print b:",b)

w_list.append(a)
print(f"first list = ",w_list)
w_list.append(b)
print(f"second list = ",w_list)
print(f"pad sequence = ", pad_sequence(w_list).shape)

print a: tensor([1, 2, 3, 4, 5])
print b: tensor([9, 8, 7, 6, 5])
first list =  [tensor([1, 2, 3, 4, 5])]
second list =  [tensor([1, 2, 3, 4, 5]), tensor([9, 8, 7, 6, 5])]
pad sequence =  torch.Size([5, 2])


In [15]:
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        label_list.append(label_transform(_label)) #label 0,1
        processed_text = torch.tensor(text_transform(_text)) #word의 index값을 list에서 Tensor 형태로
        text_list.append(processed_text)#list에 Tensoo입력
    
    return torch.tensor(label_list), pad_sequence(text_list, padding_value = 3) ##pad_sequence padding_value = <pad> 번호
        

In [19]:
train_iter = IMDB(split = 'train')
train_dataloader = DataLoader(list(train_iter), batch_size = 8, shuffle = True,
                             collate_fn  = collate_batch)

In [44]:
list(train_iter)[1][0]

'pos'

In [35]:
for label, data in train_dataloader:
    print(f"label = {label}, data = {data}")
    break

label = tensor([1, 1, 0, 0, 1, 1, 1, 0]), data = tensor([[   1,    1,    1,  ...,    1,    1,    1],
        [  15,   15,  179,  ..., 1713,  520,   96],
        [  35,  248,   92,  ...,    6, 1578,    6],
        ...,
        [   3,    3,    3,  ...,    3,  182,    3],
        [   3,    3,    3,  ...,    3,    5,    3],
        [   3,    3,    3,  ...,    3,    2,    3]])


In [33]:
a = torch.ones(25,)
b = torch.ones(22,)
c = torch.ones(15,)
pad_sequence([a, b, c]).size()

torch.Size([25, 3])

In [34]:
import random

train_iter = IMDB(split='train')
train_list = list(train_iter)
batch_size = 8  # A batch size of 8

def batch_sampler():
    indices = [(i, len(tokenizer(s[1]))) for i, s in enumerate(train_list)]
    random.shuffle(indices)
    pooled_indices = []
    # create pool of indices with similar lengths 
    for i in range(0, len(indices), batch_size * 100):
        pooled_indices.extend(sorted(indices[i:i + batch_size * 100], key=lambda x: x[1]))

    pooled_indices = [x[0] for x in pooled_indices]

    # yield indices for current batch
    for i in range(0, len(pooled_indices), batch_size):
        yield pooled_indices[i:i + batch_size]

bucket_dataloader = DataLoader(train_list, batch_sampler=batch_sampler(),
                               collate_fn=collate_batch)

print(next(iter(bucket_dataloader)))

(tensor([1, 0, 1, 1, 1, 1, 0, 0]), tensor([[    1,     1,     1,     1,     1,     1,     1,     1],
        [  537,    79,    16,    15,    59,   423,     8,    48],
        [  432,    15,    23,   428,   172,  2846,   667,    19],
        [   21,    31,   291,   457,    42,   199,  1506,     8],
        [    4,    22,   187,    16,   302, 10590,     9,  3690],
        [  877,   368,   290,    23,     6,    64,     0,    59],
        [    5,     6,   182,     5,    45,    83,     6,   298],
        [ 8948,     7,     6,    13, 10674,     5,     0,   252],
        [   24,    46,    97,  1807,     6,   124,     6,  1190],
        [ 2592,   707,  1243,    39,   175,    19,  8387,     4],
        [    5,     6,     4,  1360,     6,    64,     0,    25],
        [   97,    15,   862,     7,     7,    59,     6,    27],
        [   60,    69,   899,   440,   186,     5,     0, 10987],
        [   28,    35,    33,   711,     8,    73,  3678,     0],
        [  150,  2491,     7,    17,  129

In [40]:
from torchtext.datasets import IMDB
from torchtext.data import to_map_style_dataset
train_iter = IMDB(split='train')
train_dataset = to_map_style_dataset(from_iter_vocab)

TypeError: __getitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: torchtext._torchtext.Vocab, arg0: str) -> int

Invoked with: <torchtext._torchtext.Vocab object at 0x7f5f933a3d70>, 0

In [39]:
for label, data in train_dataset:
    print(f"label = {label}, data = {data}")
    break

label = pos, data = Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn't really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I'd have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.
